In [8]:
import time
import numpy as np
import pandas as pd
import json
import seaborn as sb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import seaborn as sb

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import pickle

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import cluster_funcs as cf
import re

ModuleNotFoundError: No module named 'cluster_funcs'

In [5]:
tutor_df = pd.read_pickle("./model_base_data.pkl")
tutor_df.head()

,qual_encoded,background_check,num_subjects,hours_tutoring,edu_length,mean_hours,state_cb,qual_encoded_cb
1456,1,1,10,2849.0,7,0.458333,3.779692,3.913101
2049,1,0,25,3.0,7,0.416667,3.737146,3.913101
644,1,1,80,2724.0,7,0.291667,3.779692,3.913101
346,0,0,11,538.0,7,0.160714,3.691904,3.744277
396,0,0,10,0.0,4,0.416667,3.691904,3.744277


In [6]:
tutor_df.iloc[1]

qual_encoded         1.000000
background_check     0.000000
num_subjects        25.000000
hours_tutoring       3.000000
edu_length           7.000000
mean_hours           0.416667
state_cb             3.737146
qual_encoded_cb      3.913101
Name: 2049, dtype: float64

In [17]:
loaded_model = pickle.load(open('model_base.sav', 'rb'))
loaded_encodings = pickle.load(open('cb_encoding.sav', 'rb'))
np.exp(loaded_model.predict(tutor_df.iloc[3])[0]).round()

35.0

In [ ]:
"Do you hold any degrees? Choose from the list."
"Do you have an approved federal background check?"
"Approximately how many hours have you tutored in the past year? Enter a number."
"Which subjects do you intend to teach? Choose as many as you like from the list."
"Where do you plan to teach? Pick a state."
"How many hours each week are you available to teach?"

In [14]:
degree_list = ['Bachelors','Masters','MD','MBA','PhD','J.D.','Other']

def qual_encode(qual):
    
    qual_list = ['Masters','MBA','PhD','J.D.']
    qual_encodings = {'Masters': 1, 'MBA': 1, 'PhD': 2, 'J.D.': 3}
    if qual in qual_list:
        return qual_encodings[qual]
    else:
        return 0

In [71]:
qual1 = 'Bachelors'
qual2 = 'Masters'
qual3 = 'PhD'
all_quals = [qual1,qual2,qual3]
qual_tot = 0

for qual in all_quals:
    qual_tot += qual_encode(qual)
    
back_check = 1
hours_tutored = 200
subject_list = ['1','2','4','4','5','6','7','8','5','2']
num_subjects = len(subject_list)
edu_length = 3*len(all_quals) + 1
state = 'CA'
hours = 10
mean_hours = hours/(24*7)

In [72]:
cats = pd.Series({'state':state,'qual_encoded':qual_tot})
user_cats = loaded_encodings.transform(cats.to_frame().T).add_suffix('_cb')
user_input = pd.Series({'qual_encoded':qual_tot, 'background_check':back_check, 'num_subjects':num_subjects,
                          'hours_tutoring':hours_tutored, 'edu_length':edu_length, 'mean_hours': mean_hours})
u_input = user_input.to_frame().T.join(user_cats)
u_input

,qual_encoded,background_check,num_subjects,hours_tutoring,edu_length,mean_hours,state_cb,qual_encoded_cb
0,3.0,1.0,10.0,200.0,10.0,0.059524,3.871119,4.277966


In [73]:
np.exp(loaded_model.predict(u_input.values))[0].round()

52.0

In [5]:
def qual_encode(qual):
    
    qual_list = ['Masters','MBA','PhD','J.D.']
    qual_encodings = {'Masters': 5, 'MBA': 5, 'PhD': 20, 'J.D.': 30}
    if qual in qual_list:
        return qual_encodings[qual]
    else:
        return 0
    
def get_exp(bio):
    
    if 'years' in bio:
        yrs = np.array([int(i) for i in re.findall('\d+', bio)])
        if len(yrs[yrs<50])>0:
            return max(yrs[yrs<50])
        else:
            return 0
    else:
        return 0

In [6]:
dbname = 'tutor_db'
username = 'codonnell'
pswd = '7883511'

con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

# query:
sql_query = """
SELECT * FROM tutor_main
"""
tutor_data = pd.read_sql_query(sql_query,con)
tutor_data = tutor_data.drop_duplicates(subset='tut_id')
tutor_data['log_hourly_rate'] = np.log(tutor_data['hourly_rate'])
tutor_data['qual_encoded'] = tutor_data.apply(lambda x: qual_encode(x['qual_1']) + qual_encode(x['qual_2']), axis =1) 
tutor_data.head()

,index,description,hourly_rate,bio,background_check,tut_id,photo,rating_count,rating_av,review_count,...,sunday_hours,monday_hours,tuesday_hours,wednesday_hours,thursday_hours,friday_hours,saturday_hours,mean_hours,log_hourly_rate,qual_encoded
0,0,Fun Experienced Tutor!,60,"['Hi!', ""I'm an experienced tutor with a sense...",1,7269032,1,269,4.98885,15.0,...,0.583333,0.416667,0.583333,0.375000,0.375000,0.583333,0.583333,0.500000,4.094345,5
1,1,"Math, Writing and Econ Specialist, 8+ Years Tu...",75,"['From Algebra to MLA, Middle School to MBA, I...",1,7277990,1,338,4.93195,18.0,...,0.500000,0.541667,0.583333,0.583333,0.583333,0.500000,0.333333,0.517857,4.317488,5
2,2,Top-Rated Test Prep (SAT / ACT / ISEE / HSPT) ...,95,"[""I am an experienced (3000+ hours) tutor cert...",1,7611743,1,1046,4.94349,74.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.553877,0
3,3,Congratulations to my students entering Harvar...,95,['MARCH 2020: IN ORDER TO ASSIST MORE STUDENTS...,1,7614535,1,1602,4.97940,32.0,...,0.000000,0.375000,0.541667,0.541667,0.250000,0.416667,0.375000,0.357143,4.553877,5
4,13,"Patient, Knowledgeable and Effective Spanish T...",125,['I am a high school AP Spanish Language teach...,1,8022035,1,67,4.98507,12.0,...,0.041667,0.166667,0.083333,0.125000,0.125000,0.083333,0.000000,0.089286,4.828314,0


In [3]:
sql_query = """
SELECT * FROM tutor_subjects
"""
tutor_subjects = pd.read_sql_query(sql_query,con)
tutor_subjects.head()

,index,tutor_ids,subjects
0,0,7269032,Precalculus
1,1,7269032,French
2,2,7269032,European History
3,3,7269032,Calculus
4,4,7269032,Spanish


In [7]:
ivys = 'Brown|Columbia|Dartmouth|Harvard|Cornell|Pennsylvania|Princeton|Yale'

tutor_data['ivy_1e'] = tutor_data['inst_1'].str.contains(ivys).astype(int)
tutor_data['ivy_2e'] = tutor_data['inst_2'].str.contains(ivys).astype(int)
tutor_data['ivy_3e'] = tutor_data['inst_3'].str.contains(ivys).astype(int)
tutor_data['ivy_tot'] = tutor_data['ivy_1e']+tutor_data['ivy_2e']+tutor_data['ivy_3e']

tutor_data['state_1e'] = tutor_data['inst_1'].str.contains('State').astype(int)
tutor_data['state_2e'] = tutor_data['inst_2'].str.contains('State').astype(int)
tutor_data['state_3e'] = tutor_data['inst_3'].str.contains('State').astype(int)
tutor_data['state_tot'] = tutor_data['state_1e']+tutor_data['state_2e']+tutor_data['state_3e']
tutor_data['experience'] = tutor_data['bio'].apply(get_exp)

NameError: name 're' is not defined